In [1]:
!pip3 install geograpy3

In [2]:
##Import
import nltk

nltk.downloader.download('maxent_ne_chunker')
nltk.downloader.download('words')
nltk.downloader.download('treebank')
nltk.downloader.download('maxent_treebank_pos_tagger')
nltk.downloader.download('punkt')
import re
import random
from dateutil.parser import parse

from geograpy import places

# Create templates
bot_template = "JOHNNY : {0}"
user_template = "USER : {0}"
name = 0
st = 0
memo = []
welcome = 0

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/deptinfo/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /home/deptinfo/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package treebank to
[nltk_data]     /home/deptinfo/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     /home/deptinfo/nltk_data...
[nltk_data]   Package maxent_treebank_pos_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/deptinfo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
from geograpy import places

def is_destination(mess):
    pc = places.PlaceContext([mess])
    pc.set_countries()
    if len(pc.countries) > 0 :
        return True
    else :
        pc = places.PlaceContext([mess])
        pc.set_regions()
        if len(pc.regions) > 0:
            return True
        else :
            return False

In [4]:
def to_capitalize(message):
    capitalized_message = " ".join([
       word.capitalize()
       for word in message.split(" ")
    ])
    return capitalized_message

In [5]:
def is_date(string, fuzzy=False):
    try: 
        parse(string, fuzzy=fuzzy)
        return True

    except ValueError:
        return False

In [6]:
# Define a function that sends a message to the bot: send_message
def send_message(mess):
    global user_template
    # Print user_template including the user_message
    print(user_template.format(mess))
    # Get the bot's response to the message
    response = respond(mess)
    # Print the bot template including the bot's response.
    print(bot_template.format(response))

In [7]:
def responsebot(message):
    global start
    global name
    global st
    global user_template
    message = message.lower()
    if len(memo) == 0 :
        if 'from' in message:
            tab = message.split("from ")
            if is_destination(to_capitalize(tab[1])):
                memo.append(tab[1].upper())
                return "Ok ! You leaving from " + tab[1].upper() + " and where are you going ?"
            else :
                return "Can you repeat ? I don't understand.. I do not know this place. "
        else :
            return "Sorry, I don't understand your answer, can you answer to my question please ? So, where are you from ?"
            
    elif len(memo) == 1 :
        if 'to' in message:
            tab = message.split("to ")
            if is_destination(to_capitalize(tab[1])):
                memo.append(tab[1].upper())
                return "To " + tab[1].upper() + " Okay ! What date do you want to leave ?"
            else :
                return "Can you repeat ? I don't understand..I do not know this place."
        else : 
            return "Sorry, I don't understand your answer, can you answer to my question please ? So, where are you going ?"
    elif len(memo) == 2 :
        if 'in' in message:
            tab = message.split("in ")
            if is_date(tab[1]):
                memo.append(tab[1].upper())
                return "In the " + tab[1].upper() + " ! Is it a one-way trip?"
            else :
                return "Can you repeat ? I don't understand.."
        else :
            if is_date(message):
                memo.append(message.upper())
                return "In the " + message.upper() + " ! Is it a one-way trip?"
            else :
                return "Can you repeat ? I don't understand.."
    elif len(memo) == 3 :
        if 'yes' in message :
            #start += 1
            memo.append("fin")
            return "To summarize you leaving from " + memo[0] + " to " + memo[1]+ " on "+ memo[2]+ " ?"
        elif 'no' in message :
            return "Okay, tell me what date do you want to return ?"
        elif "in" in message :
            tab = message.split("in")
            memo.append(tab[1].upper())
            return "To summarize you leaving from " + memo[0] + " to " + memo[1]+ " on "+ memo[2] + " returning on " + memo[3] + "?"
        else :
            if is_date(message):
                memo.append(message.upper())
                return "To summarize you leaving from " + memo[0] + " to " + memo[1]+ " on "+ memo[2] + " returning on " + memo[3]

    elif len(memo) == 4 :
        if 'yes' in message :
            start = 0
            name = 0
            st = 0
            del memo[:]
            user_template = "USER : {0}"
            return "Bon voyage ! The flight was booking"
        if 'no' in message :
            start = 1
            del memo[:]
            return "Okay, let's try again"

In [8]:
namerules = {
    "I'm (.*)": 
    [
        "{0}, Nice to meet you. I'm Johnny ! "
    ],
    "My name is (.*)": 
    [
        "{0}, Nice to meet you. I'm Johnny ! "
    ]
}

def changename(messages):
    global user_template
    global name
    response, phrase = "default", None
    for pattern, rules_responses in namerules.items():
        # Create a match object
        match = re.search(pattern,messages)

        if match is not None:
            # Choose a random response
            response = random.choice(rules_responses)
            if '{0}' in response:
                phrase = match.group(1)
                names = phrase
                user_template = names.upper() +" : {0}"
                name += 1
        if response == "default":
            response = "Tell me your name !!"
    return phrase, response

In [9]:
start = 0
def respond(message):
    phrase = ""
    global start
    global name
    global st
    if name == 0 and st == 0:
        phrase = "Hi ! Tell me your name please"
        st += 1
    elif name == 0 and st == 1 :
        phrase,response = changename(message)
        phrase = response.format(phrase)
    if name ==1 :
        if start == 0:
            phrase = phrase + "I am your assistant to help you with the purchase of a flight ticket. First, What city are you leaving from ?"
            start += 1
        elif start == 1 :
            phrase = responsebot(message)
    return phrase

In [10]:
send_message('Hi')
send_message("I'm SpongeBob")
send_message("test de phrase qui n'a aucun sens ")
send_message("I'm leaving from Nice")
send_message("I'm going to Antibes")
send_message("12/11/2020")
send_message("yes")
send_message("Yes")


USER : Hi
JOHNNY : Hi ! Tell me your name please
USER : I'm SpongeBob
JOHNNY : SpongeBob, Nice to meet you. I'm Johnny ! I am your assistant to help you with the purchase of a flight ticket. First, What city are you leaving from ?
SPONGEBOB : test de phrase qui n'a aucun sens 
JOHNNY : Sorry, I don't understand your answer, can you answer to my question please ? So, where are you from ?
SPONGEBOB : I'm leaving from Nice
JOHNNY : Ok ! You leaving from NICE and where are you going ?
SPONGEBOB : I'm going to Antibes
JOHNNY : To ANTIBES Okay ! What date do you want to leave ?
SPONGEBOB : 12/11/2020
JOHNNY : In the 12/11/2020 ! Is it a one-way trip?
SPONGEBOB : yes
JOHNNY : To summarize you leaving from NICE to ANTIBES on 12/11/2020 ?
SPONGEBOB : Yes
JOHNNY : Bon voyage ! The flight was booking


In [11]:
print("-----------------------------------------------------------------")
send_message('Hi')
send_message("My name is yasmin")
send_message("I'm leaving from New York")
send_message("I'm going to Antibes")
send_message("12 jan 2020")
send_message("No")
send_message("22 january 2020")
send_message("yes")

-----------------------------------------------------------------
USER : Hi
JOHNNY : Hi ! Tell me your name please
USER : My name is yasmin
JOHNNY : yasmin, Nice to meet you. I'm Johnny ! I am your assistant to help you with the purchase of a flight ticket. First, What city are you leaving from ?
YASMIN : I'm leaving from New York
JOHNNY : Ok ! You leaving from NEW YORK and where are you going ?
YASMIN : I'm going to Antibes
JOHNNY : To ANTIBES Okay ! What date do you want to leave ?
YASMIN : 12 jan 2020
JOHNNY : In the 12 JAN 2020 ! Is it a one-way trip?
YASMIN : No
JOHNNY : Okay, tell me what date do you want to return ?
YASMIN : 22 january 2020
JOHNNY : To summarize you leaving from NEW YORK to ANTIBES on 12 JAN 2020 returning on 22 JANUARY 2020
YASMIN : yes
JOHNNY : Bon voyage ! The flight was booking
